In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation

# importe le simulateur depuis simulateur.py
from simulateur import *

# importe l'algorithme RRT*
from PythonRobotics.PathPlanning.RRTStar.rrt_star import RRTStar

### Préparer la carte

In [2]:
# Cell Dimension: 633, 1149

# importing the map and droping the last empty column
df = pd.read_csv('RDCmod3.txt', header=None)
df = df.drop(columns=[633])

# droping columns and rows with 0
df = df.loc[:, (df != 0).any(axis=0)]
df = df.loc[(df!=0).any(axis=1)]

#normalizing the dataframe
df_norm = -df.copy()
for column in df_norm.columns:
    df_norm[column] = df_norm[column] / df_norm[column].abs().max()
    
#Rounding the dataframe
df_rounded = df_norm.round(0).astype(int)
df_rounded = df_rounded + 1
df_rounded.replace(2,1, inplace=True)

#Exporting the final map
df_rounded.to_csv("mapnorm.csv",sep=';', header=False, index=False)

### Simulation

In [3]:
#Création de la pièce et du chien
room = DogRoom('mapnorm.csv')
# room = DogRoom('map01.csv')
dog = Dog(room)
# human = Human(room, speed = 10)

In [5]:
#plotting the map with imshow

%matplotlib notebook
def showmap():
    dog_x, dog_y = dog.x, dog.y
    height, width = room.imgdf.shape
    # human_x, human_y = human.x, human.y

    fig,ax = plt.subplots(1,figsize=(7,7))
    fig = plt.plot(dog_x, dog_y, 's')
    # fig = plt.plot(human_x, human_y, '0')
    
    ax.imshow(room.imgdf, origin='lower', interpolation='none', extent=(0, width, 0, height))
    plt.gca().invert_yaxis()
    plt.show()
    
# Draw final path
def showpath(room, dog, obstacle, path): 
    %matplotlib inline
    fig,ax = plt.subplots(1,figsize=(7,7))
    
    width, height = room.largeur, room.longueur

    plt.plot([x for (x, y) in path], [y for (x, y) in path], 'r--') #Plotting the path
    plt.plot([x for (x, y) in path], [y for (x, y) in path], 'o', color='green') #Plotting the path
    plt.plot(dog.x, dog.y, 'o', color='blue')
    plt.plot(goal[0],goal[1],'x', color='orange')

    for (x, y, radius) in obstacle:
        cir = plt.Circle((x, y), radius, color='r', alpha=0.5, fill=True)
        ax.add_patch(cir)
    ax.imshow(room.imgdf, origin='lower', interpolation='none', extent=(0, width, 0, height))
    plt.gca().invert_yaxis()
    plt.show()

def generatepath(goal, obstacle):
    # Set Initial parameters
    rrt_star = RRTStar(
        start=[dog.x, dog.y],
        goal=goal,
        rand_area=[-2, 15],
        obstacle_list=obstacle_list,
        expand_dis=1,
        max_iter=300,
        search_until_max_iter=True)
    path = rrt_star.planning(animation=False)

    if path is None:
        print("Cannot find path")
    else:
        print("found path!!")
        showpath(room, dog, obstacle, path)
    return path

def MoveToPoint(dog, x, y):
    
#     unit_vector_x = [1,0]
  
    #info sur le chien
    pos = np.array([dog.x, dog.y])
    curr_angle = dog.rot

    #destination
    goal = np.array([x,y])
    mov_vector = goal - pos
    
    dist = np.linalg.norm(mov_vector)

    dot = np.dot(mov_vector, [1,0])
    det = np.linalg.det(np.array([mov_vector, [1,0]]))
    angle = np.degrees(np.arctan2(det, dot)) #problème d'angle hors intervalle pour le robot
    rot = angle - curr_angle
    
    print("Current X: {}, Current Y: {}".format(pos[0], pos[1]))
    print("Goal X: {}, Goal Y: {}".format(x,y))
    print("Angle absolu {}".format(angle))
    print("Angle relatif {}".format(rot))
    print("Distance à parcourir {}".format(dist))
    print("--------------")

    #fonction du simuateur
    dog.rotate(rot)
    dog.forward(dist)
    
    
#Using the path to move the dog
def MoveUsingPath(dog, path):
    keypoints = path.copy()
    keypoints.reverse()

    for x,y in keypoints[1:]:
        MoveToPoint(dog, x, y)
    showmap()
    return True

In [4]:
testdf = room.imgdf.copy()
obstacle = []
radius = 1.2
for i in range(len(testdf)):
    for j in range(len(testdf.iloc[i])):
        if testdf.iloc[i][j]==1:
            obstacle.append((j+radius/2,i+radius/2,radius))

In [8]:
room.plotmap()

Dimension: 682 x 253


<IPython.core.display.Javascript object>

In [8]:
show_animation = True
obstacle_list = obstacle
# goal = [8, 3]
goal = [140, 340]
# generatepath(goal, obstacle)


In [14]:
dog.x, dog.y = 15, 35

In [ ]:
path = generatepath(goal, obstacle)
MoveUsingPath(dog, path)

Iter: 0 , number of nodes: 1
Iter: 1 , number of nodes: 2
Iter: 2 , number of nodes: 3
Iter: 3 , number of nodes: 4
Iter: 4 , number of nodes: 5
Iter: 5 , number of nodes: 6
Iter: 6 , number of nodes: 7
Iter: 7 , number of nodes: 8
Iter: 8 , number of nodes: 9
Iter: 9 , number of nodes: 10
Iter: 10 , number of nodes: 11
Iter: 11 , number of nodes: 12
Iter: 12 , number of nodes: 13
Iter: 13 , number of nodes: 14
Iter: 14 , number of nodes: 15
Iter: 15 , number of nodes: 16
Iter: 16 , number of nodes: 17
Iter: 17 , number of nodes: 18
Iter: 18 , number of nodes: 19
Iter: 19 , number of nodes: 20
Iter: 20 , number of nodes: 21
Iter: 21 , number of nodes: 22
Iter: 22 , number of nodes: 23
Iter: 23 , number of nodes: 24
Iter: 24 , number of nodes: 25
Iter: 25 , number of nodes: 26
Iter: 26 , number of nodes: 27
Iter: 27 , number of nodes: 28
Iter: 28 , number of nodes: 29
Iter: 29 , number of nodes: 30
Iter: 30 , number of nodes: 31
Iter: 31 , number of nodes: 32
Iter: 32 , number of nodes:

Iter: 256 , number of nodes: 257
Iter: 257 , number of nodes: 258
Iter: 258 , number of nodes: 259
Iter: 259 , number of nodes: 260
Iter: 260 , number of nodes: 261
Iter: 261 , number of nodes: 262
Iter: 262 , number of nodes: 263
Iter: 263 , number of nodes: 264
Iter: 264 , number of nodes: 265
Iter: 265 , number of nodes: 266
Iter: 266 , number of nodes: 267
Iter: 267 , number of nodes: 267
Iter: 268 , number of nodes: 267
Iter: 269 , number of nodes: 267
Iter: 270 , number of nodes: 268
Iter: 271 , number of nodes: 268
Iter: 272 , number of nodes: 268
Iter: 273 , number of nodes: 268
Iter: 274 , number of nodes: 268
Iter: 275 , number of nodes: 268
Iter: 276 , number of nodes: 268
Iter: 277 , number of nodes: 269
Iter: 278 , number of nodes: 270
Iter: 279 , number of nodes: 270
Iter: 280 , number of nodes: 270
Iter: 281 , number of nodes: 271
Iter: 282 , number of nodes: 271
Iter: 283 , number of nodes: 272
Iter: 284 , number of nodes: 272
Iter: 285 , number of nodes: 272
Iter: 286 